# SageMaker Model Training and Prediction
## Introduction
__"Leverage SageMaker for Notebook --> Production Testing"__

---
## 1 - Model Training
### Permissions and Environmental Variables

In [ ]:
# Import libraries
from __future__ import print_function
import os
import io
import boto3
import sagemaker
import h5py
import json
import numpy as np
import mxnet as mx
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

# Confifuree SageMaker
sagemaker_session = sagemaker.Session()
role = get_execution_role()

## Input Data Preparation

In [ ]:
# Create local repository for Numpy Arrays
if not os.path.exists('tmp'): os.mkdir('tmp')

# Load the Training and Testing dataset
dataset = h5py.File('datasets/datasets.h5', 'r')

# Save the Dataset as Numpy Arrays
np.save('tmp/train_X.npy', np.array(dataset['train_set_x'][:]))
np.save('tmp/train_Y.npy', np.array(dataset['train_set_y'][:]))
np.save('tmp/test_X.npy', np.array(dataset['test_set_x'][:]))
np.save('tmp/test_Y.npy', np.array(dataset['test_set_y'][:]))

# Upload the Training and Testing Data to S3
inputs = sagemaker_session.upload_data(path='tmp', key_prefix='training_input')
bucket = inputs.split('/')[2]
# Create a new Job name for current run
job_name = 'sagemaker-train-v0.1'

## Create the Estimator

In [ ]:
mxnet_estimator = MXNet(
    'model.py',
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    output_path='s3://'+bucket,
    hyperparameters={
        'epochs': 2500,
        'optmizer': 'sgd',
        'learning_rate': 0.0075,
        'batch_size': 64
    }
)

## Train the Model

In [ ]:
mxnet_estimator.fit(inputs, job_name=job_name)

---
## 2 - Training Analysis

In [ ]:
# Download and uncompress output results from model training
import tarfile, datetime
import matplotlib.pyplot as plt
%matplotlib inline
s3 = boto3.resource('s3')
s3.Bucket(bucket).download_file(job_name+'/output/output.tar.gz', './output.tar.gz')
tarfile.open('./output.tar.gz').extractall()
with open('results.json') as j:
    data = json.load(j)#, object_pairs_hook=OrderedDict)

# Format data for plotting
costs = []
for key, value in sorted(data.iteritems(), key=lambda (k,v): (v, k)):
    if 'epoch' in key:
        costs.append(value)
    elif 'Start' in key:
        start = datetime.datetime.strptime(value, "%Y-%m-%d %H:%M:%S.%f")
    elif 'End' in key:
        end = datetime.datetime.strptime(value, "%Y-%m-%d %H:%M:%S.%f")
    elif 'val_accuracy' in key:
        val_accuracy = value
    elif 'train_accuracy' in key:
        train_accuracy = value
delta = end - start
costs.reverse()
print("Model Training Time: {} Minute(s)".format(int(delta.total_seconds() / 60)))
print("Training Accuracy: {}".format(train_accuracy))
print("Validation Accuracy: {}".format(val_accuracy))

# Plot the results
plt.rcParams['figure.figsize'] = (11.0, 10.0)
plt.plot(costs)
plt.grid(True, which="both")
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.title("Learning Curve")
plt.show;

---
## 3 - Prediciton Analysis

In [ ]:
predictor = mxnet_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Grumpy Cat

In [ ]:
from skimage import transform
# Get Classes
tmp_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
classes = np.array(tmp_dataset['list_classes'][:])
# Display and predict against grumpy cat image
my_image = "grumpy_cat.jpeg"
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))

# Run the image against the SageMaker endpoint 
prediction = int(predictor.predict(image.tolist()))

# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(prediction)+"\nThe trained model predicts a \""+classes[prediction]+"\" picture.")

### Gargouille

In [ ]:
# Display and predict against gargouille image
my_image = "gargouille.jpeg"
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))

# Run the image against the SageMaker endpoint 
prediction = int(predictor.predict(image.tolist()))

# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(prediction)+"\nThe trained model predicts a \""+classes[prediction]+"\" picture.")

### Eiffle Tower (Day)

In [ ]:
# Display and predict against Eiffle Tower image
my_image = "my_image.jpeg"
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))

# Run the image against the SageMaker endpoint 
prediction = int(predictor.predict(image.tolist()))

# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(prediction)+"\nThe trained model predicts a \""+classes[prediction]+"\" picture.")

### Eiffle Tower (Dusk)

In [ ]:
# Display and predict against Eiffle Tower image
my_image = "la_defense.jpeg"
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))

# Run the image against the SageMaker endpoint 
prediction = int(predictor.predict(image.tolist()))

# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(prediction)+"\nThe trained model predicts a \""+classes[prediction]+"\" picture.")

### White Cat

In [ ]:
# Display and predict against white cat image
my_image = "my_image2.jpeg"
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))

# Run the image against the SageMaker endpoint 
prediction = int(predictor.predict(image.tolist()))

# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(prediction)+"\nThe trained model predicts a \""+classes[prediction]+"\" picture.")

### Cat in Iran

In [ ]:
# Display and predict against cat in iran image
my_image = "cat_in_iran.jpeg"
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))

# Run the image against the SageMaker endpoint 
prediction = int(predictor.predict(image.tolist()))

# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(prediction)+"\nThe trained model predicts a \""+classes[prediction]+"\" picture.")